In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [3]:
df = pd.read_csv('~/data/training_data.csv')
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [4]:
sens = 2

In [25]:
X = df_normal.iloc[:,sens].values
X = hankel.fit(X,30,0.5)
X=X.T # size--> 1069X1800
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis_normal = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radiis_normal.append(np.sqrt(var2))

4
r: 13
r: 14
r: 12
r: 12


In [26]:
radiis_normal

[array([0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 ,
        0.5281653 , 0.52816341, 0.52816417, 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 ,
        0.81450908, 0.5281653 , 0.5281653 , 0.52814836, 0.5281653 ,
        0.5281653 , 0.5281653 , 0.52801577, 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.51208753, 0.5281653 , 0.5281653 ,
        0.52441103, 0.55077531, 0.33023003, 0.5281653 , 0.5281653 ,
        0.94929606, 0.51371159, 0.5281653 , 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.5281653 , 0.52539288, 0.57243067,
        0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 ,
        0.6611072 , 0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.74795624, 0.68134761, 0.5281653 ,
        0.80347161, 0.67767177, 0.51476658, 0.5281653 , 0.5281653 ,
        0.5281653 , 0.5281653 , 0.5281653 , 0.56

In [27]:
X_attack = df_attack.iloc[:,sens].values
X_att = hankel.fit(X_attack,30,0.5)
X_att=X_att.T # size--> X1800
radiis_attack = []
print(X_att.shape)
for i in range(optimal_k):
    cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
    var1=np.square(cluster_-centers[i])
    var2=np.matmul(weights[i],var1.T)
    radiis_attack.append(np.sqrt(var2))

(8, 1800)


In [28]:
radiis_attack

[array([2.04460041, 1.89138041, 2.33470238, 1.89481133, 0.56944744,
        0.51058033, 1.08030554, 1.88842011]),
 array([0.89838727, 1.21319073, 0.91170066, 1.24475302, 1.4859964 ,
        1.42592019, 1.45372831, 1.04257776]),
 array([1.70253107, 1.22080838, 1.14771584, 1.87751236, 1.52459381,
        1.34596921, 0.99339345, 0.96382726]),
 array([1.07778941, 1.18992323, 1.40843227, 0.82662075, 1.28146131,
        1.17174894, 1.30176411, 1.75330145])]

In [29]:
# def calc_radius(X,kmeans,optimal_k):
#     radiis = []
#     weights = []
#     centers = []
#     topR =[ ]
#     cluster_V = []
#     for i in range(optimal_k):
#         cluster_points = X[np.where(kmeans.labels_ == i)[0]]
#         r = rank.fit(X)
#         print('r: ',r)
#         topR.append(r)
#         U,Sigma,VT = np.linalg.svd(cluster_points)
#         V = VT.T
#         cluster_V.append(V[:,:r])
#         cluster_points = np.matmul(cluster_points,V[:,:r])
#         print(cluster_points.shape)
#         weight,center = meepc.fit(cluster_points)
#         weights.append(weight)
#         centers.append(center)
#         var1=np.square(cluster_points-center)
#         var2=np.matmul(weight,var1.T)
#         # max_radius=np.amax(var2)
#         radiis.append(np.sqrt(var2))
#     return radiis,weights,centers,cluster_V,topR

In [30]:
# X_n = df_normal.iloc[:,2].values
# X_n = hankel.fit(X_n,10,0.5)
# X_n = X_n.T
# kmeans,optimal_k = cluster.fit(X_n)
# print(optimal_k)
# radiis_normal = []
# for i in range(optimal_k):
#     cluster_points = X_n[np.where(kmeans.labels_ == i)[0]]
#     r = rank.fit(X_n)
#     print('r: ',r)
#     U,Sigma,VT = np.linalg.svd(cluster_points)
#     V = VT.T
#     cluster_points = np.matmul(cluster_points,V[:,:r])
#     print(cluster_points.shape)
#     weight,center = meepc.fit(cluster_points)
#     var1=np.square(cluster_points-center)
#     var2=np.matmul(weight,var1.T)
#     radiis_normal.append(np.sqrt(var2))
# # radiis_normal,weights_normal,centers_normal,cluster_V_normal,topR_normal = calc_radius(X_n,kmeans,optimal_k)
# # radiis_normal

In [31]:
# X_a = df_attack.iloc[:,2].values
# X_a = hankel.fit(X_a,10,0.5)
# X_a = X_a.T
# kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
# kmeans.fit(X_a)
# radiis_attack,weights_attack,centers_attack,cluster_V_attack,topR_attack = calc_radius(X_a,kmeans,optimal_k)
# radiis_attack

In [32]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    print(indices[idx])
    return radiis[indices[idx]]

In [33]:
threshold_clusters = [0]*optimal_k
for i in range(optimal_k):
    threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])
threshold_clusters

161
239
16
9


[1.0002201982034264, 1.0014734114254868, 1.01316066563225, 1.0003705943179206]

In [34]:
df_test=pd.read_csv('~/data/testdata.csv')
y_actual=df_test.iloc[:,-1]
df_test=df_test.iloc[:,sensors]

In [35]:
labels=hankel.fit(np.array(y_actual),30,0.5)
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([267, 268, 269, 272, 273])

In [36]:
X_test=df_test.iloc[:,sens].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

(274, 1800)


In [37]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[1.91183702, 1.23307772, 1.88980583, 0.82753116],
       [0.59516904, 1.45744475, 1.52712792, 1.30231661],
       [0.5280319 , 1.44444121, 1.38266788, 1.18354432],
       ...,
       [1.22403106, 1.01382955, 0.40179398, 1.33924728],
       [1.85588754, 0.42158724, 1.09004837, 0.89789537],
       [1.93499066, 0.50912986, 1.08380835, 0.88750514]])

In [38]:
check_anamoly=threshold_clusters-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[-0.91161682, -0.2316043 , -0.87664516,  0.17283943],
       [ 0.40505116, -0.45597134, -0.51396725, -0.30194601],
       [ 0.4721883 , -0.4429678 , -0.36950722, -0.18317372],
       ...,
       [-0.22381086, -0.01235614,  0.61136668, -0.33887669],
       [-0.85566734,  0.57988617, -0.0768877 ,  0.10247522],
       [-0.93477046,  0.49234356, -0.07064768,  0.11286546]])

In [39]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]

In [43]:
for i in [[267, 268, 269, 272, 273]]:
    print(distances_arr[i])

[[1.65116706 1.00699797 0.50632471 1.56746658]
 [2.5804378  0.45409046 1.53070556 1.28502463]
 [2.39093372 0.77822819 1.67163317 1.28065739]
 [1.85588754 0.42158724 1.09004837 0.89789537]
 [1.93499066 0.50912986 1.08380835 0.88750514]]


In [42]:
for i in range(274):
  if i not in positive_indices:
    print(i)

88
93
94
147
151


In [21]:
# array([803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])